In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import preprocessing, metrics
import lightgbm as lgb

import warnings
warnings.filterwarnings("ignore")

In [2]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

### data

In [3]:
# исторические данные о ежедневных продажах единицы продукции по продукту и магазину [d_1 - d_1913]
train = pd.read_csv('sales_train_validation.csv')
# Includes sales [d_1 - d_1941] (labels used for the Public leaderboard)
# лейблы для следующих 28 дней для прогноза
train_addit = pd.read_csv('sales_train_evaluation.csv')
# Содержит информацию о датах продажи товаров
calendar = pd.read_csv('calendar.csv')
calendar = reduce_mem_usage(calendar)
# Содержит информацию о цене товаров, проданных в каждом магазине, и дате.
sell_prices = pd.read_csv('sell_prices.csv')
sell_prices = reduce_mem_usage(sell_prices)
submission = pd.read_csv('sample_submission.csv')

Mem. usage decreased to  0.12 Mb (41.9% reduction)
Mem. usage decreased to 130.48 Mb (37.5% reduction)


In [4]:
train_table = pd.melt(train, id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], 
        var_name = 'day', value_name = 'demand')
train_table = reduce_mem_usage(train_table)

Mem. usage decreased to 3226.27 Mb (9.4% reduction)


In [5]:
print(train.shape)
print(train_table.shape)
train_table.head()

(30490, 1919)
(58327370, 8)


,id,item_id,dept_id,cat_id,store_id,state_id,day,demand
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0


In [6]:
# seperate test dataframes
test1_rows = [row for row in submission['id'] if 'validation' in row]
test2_rows = [row for row in submission['id'] if 'evaluation' in row]
test1 = submission[submission['id'].isin(test1_rows)]
test2 = submission[submission['id'].isin(test2_rows)]

# change column names
test1.columns = ['id', 'd_1914', 'd_1915', 'd_1916', 'd_1917', 'd_1918', 'd_1919', 'd_1920', 'd_1921', 'd_1922', 'd_1923', 'd_1924', 'd_1925', 'd_1926', 'd_1927', 'd_1928', 'd_1929', 'd_1930', 'd_1931', 
                  'd_1932', 'd_1933', 'd_1934', 'd_1935', 'd_1936', 'd_1937', 'd_1938', 'd_1939', 'd_1940', 'd_1941']
test2.columns = ['id', 'd_1942', 'd_1943', 'd_1944', 'd_1945', 'd_1946', 'd_1947', 'd_1948', 'd_1949', 'd_1950', 'd_1951', 'd_1952', 'd_1953', 'd_1954', 'd_1955', 'd_1956', 'd_1957', 'd_1958', 'd_1959', 
                  'd_1960', 'd_1961', 'd_1962', 'd_1963', 'd_1964', 'd_1965', 'd_1966', 'd_1967', 'd_1968', 'd_1969']

In [7]:
# get product table
product = train[['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']].drop_duplicates()
product.head()

,id,item_id,dept_id,cat_id,store_id,state_id
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA


In [8]:
# merge with product table
test2['id'] = test2['id'].str.replace('_evaluation','_validation')
test1 = test1.merge(product, how = 'left', on = 'id')
test2 = test2.merge(product, how = 'left', on = 'id')
test2['id'] = test2['id'].str.replace('_validation','_evaluation')

In [13]:
test1 = pd.melt(test1, id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], 
                var_name = 'day', value_name = 'demand')
test2 = pd.melt(test2, id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], 
                var_name = 'day', value_name = 'demand')

In [12]:
test2 = pd.melt(test2, id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], 
                var_name = 'day', value_name = 'demand')

In [14]:
train_table['part'] = 'train'
test1['part'] = 'test1'
test2['part'] = 'test2'

In [15]:
del train

In [16]:
data = pd.concat([train_table, test1, test2], axis = 0)

In [17]:
del train_table, test1, test2

In [18]:
print(data.shape)
data.head()

(60034810, 9)


,id,item_id,dept_id,cat_id,store_id,state_id,day,demand,part
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,train
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,train
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,train
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,train
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,train


In [19]:
# delete test2 for now
data = data[data['part'] != 'test2']

In [20]:
# cut by store_id
data = data[data['store_id'] == 'TX_1']

In [21]:
# drop some calendar features
calendar.drop(['weekday', 'wday', 'month', 'year'], inplace = True, axis = 1)

In [22]:
data = pd.merge(data, calendar, how = 'left', left_on = ['day'], right_on = ['d'])
data.drop(['d', 'day'], inplace = True, axis = 1)

# get the sell price data (this feature should be very important)
data = data.merge(sell_prices, on = ['store_id', 'item_id', 'wm_yr_wk'], how = 'left')

In [23]:
data.head()

,id,item_id,dept_id,cat_id,store_id,state_id,demand,part,date,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_001_TX_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,TX_1,TX,0,train,2011-01-29,11101,NaN,NaN,NaN,NaN,0,0,0,NaN
1,HOBBIES_1_002_TX_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,TX_1,TX,0,train,2011-01-29,11101,NaN,NaN,NaN,NaN,0,0,0,NaN
2,HOBBIES_1_003_TX_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,TX_1,TX,0,train,2011-01-29,11101,NaN,NaN,NaN,NaN,0,0,0,NaN
3,HOBBIES_1_004_TX_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,TX_1,TX,1,train,2011-01-29,11101,NaN,NaN,NaN,NaN,0,0,0,4.339844
4,HOBBIES_1_005_TX_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,TX_1,TX,0,train,2011-01-29,11101,NaN,NaN,NaN,NaN,0,0,0,NaN


### features

In [24]:
nan_features = ['event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']
for feature in nan_features:
    data[feature].fillna('unknown', inplace = True)

cat = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']
for feature in cat:
    encoder = preprocessing.LabelEncoder()
    data[feature] = encoder.fit_transform(data[feature])

In [25]:
# rolling demand features
data['lag_t28'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28))
data['lag_t29'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(29))
data['lag_t30'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(30))
data['rolling_mean_t7'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(7).mean())
data['rolling_std_t7'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(7).std())
data['rolling_mean_t30'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(30).mean())
data['rolling_mean_t90'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(90).mean())
data['rolling_mean_t180'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(180).mean())
data['rolling_std_t30'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(30).std())
data['rolling_skew_t30'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(30).skew())
data['rolling_kurt_t30'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(30).kurt())

In [26]:
# price features
data['lag_price_t1'] = data.groupby(['id'])['sell_price'].transform(lambda x: x.shift(1))
data['price_change_t1'] = (data['lag_price_t1'] - data['sell_price']) / (data['lag_price_t1'])
data['rolling_price_max_t365'] = data.groupby(['id'])['sell_price'].transform(lambda x: x.shift(1).rolling(365).max())
data['price_change_t365'] = (data['rolling_price_max_t365'] - data['sell_price']) / (data['rolling_price_max_t365'])
data['rolling_price_std_t7'] = data.groupby(['id'])['sell_price'].transform(lambda x: x.rolling(7).std())
data['rolling_price_std_t30'] = data.groupby(['id'])['sell_price'].transform(lambda x: x.rolling(30).std())
data.drop(['rolling_price_max_t365', 'lag_price_t1'], inplace = True, axis = 1)

In [27]:
# time features
data['date'] = pd.to_datetime(data['date'])
data['year'] = data['date'].dt.year
data['month'] = data['date'].dt.month
data['week'] = data['date'].dt.week
data['day'] = data['date'].dt.day
data['dayofweek'] = data['date'].dt.dayofweek

In [28]:
from tsfresh import extract_features
from sklearn.feature_selection import VarianceThreshold
from tsfresh.feature_extraction import feature_calculators

from sklearn.linear_model import LinearRegression

In [29]:
np.random.seed(0)
d = data[['id', 'demand','date']].sample(1000).fillna(0)
data_ts_f = extract_features(d, column_id='id', column_sort='date', disable_progressbar=True)
data_ts_f.replace(np.inf, 0, inplace=True)

selector = VarianceThreshold(threshold=0) # удаляем константы
X_train = selector.fit_transform(data_ts_f)
X_train  = data_ts_f[data_ts_f.columns[selector.get_support(indices=True)]]
X_train = d.set_index('id').join(X_train)

In [30]:
model = LinearRegression()
# fit the model
model.fit(X_train.fillna(0).drop(['demand','date'], axis=1), X_train['demand'])
# get importance
importance = model.coef_
imp = pd.DataFrame({i:[j] for i, j in zip(X_train.columns, importance)}).T.reset_index()
imp.columns= ['name','importance']
imp['importance'] = np.abs(imp['importance'])
imp.sort_values('importance', inplace=True)
imp.tail(10)

,name,importance
71,demand__quantile__q_0.4,0.070678
72,demand__quantile__q_0.6,0.070699
73,demand__quantile__q_0.7,0.070719
31,demand__sum_of_reoccurring_data_points,0.070739
69,demand__quantile__q_0.2,0.070922
10,demand__mean_second_derivative_central,0.071145
68,demand__quantile__q_0.1,0.071207
67,demand__large_standard_deviation__r_0.45,0.071491
66,demand__large_standard_deviation__r_0.4,0.071775
32,demand__ratio_value_number_to_time_series_length,0.072059


In [ ]:
#imp.tail(10)['name'].values

In [31]:
data['feat1'] = data.groupby(['id'])['demand'].transform(lambda x: feature_calculators.ratio_value_number_to_time_series_length(x))
data['feat2'] = data.groupby(['id'])['demand'].transform(lambda x: feature_calculators.large_standard_deviation(x, 0.4))
data['feat3'] = data.groupby(['id'])['demand'].transform(lambda x: feature_calculators.large_standard_deviation(x, 0.45))
data['feat4'] = data.groupby(['id'])['demand'].transform(lambda x: feature_calculators.mean_second_derivative_central(x))
data['feat5'] = data.groupby(['id'])['demand'].transform(lambda x: feature_calculators.quantile(x, 0.1))

### model

In [32]:
# going to evaluate with the last 28 days
x_train = data[data['date'] <= '2016-03-27']
y_train = x_train['demand']
x_val = data[(data['date'] > '2016-03-27') & (data['date'] <= '2016-04-24')]
y_val = x_val['demand']
test = data[(data['date'] > '2016-04-24')]
del data

In [33]:
# define list of features
features = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'year', 'month', 'week', 'day', 'dayofweek', 
            'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 
            'snap_CA', 'snap_TX', 'snap_WI', 'sell_price', 'lag_t28', 'lag_t29', 'lag_t30', 
            'rolling_mean_t7', 'rolling_std_t7', 'rolling_mean_t30', 'rolling_mean_t90', 
            'rolling_mean_t180', 'rolling_std_t30', 'price_change_t1', 'price_change_t365', 
            'rolling_price_std_t7', 'rolling_price_std_t30', 'rolling_skew_t30', 'rolling_kurt_t30',
            'feat1', 'feat2', 'feat3', 'feat4','feat5']

In [65]:
# define random hyperparammeters
params = {
    'boosting_type': 'gbdt',
    'metric': 'rmse',
    'objective': 'regression',
    'n_jobs': -1,
    'seed': 236,
    'learning_rate': 0.1,
    'bagging_fraction': 0.75,
    'bagging_freq': 10, 
    'colsample_bytree': 0.75
}

train_set = lgb.Dataset(x_train[features], y_train)
val_set = lgb.Dataset(x_val[features], y_val)

#del x_train, y_train

In [66]:
model = lgb.train(params, train_set, num_boost_round = 2500, early_stopping_rounds = 50, 
                  valid_sets = [train_set, val_set], verbose_eval = 100)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.406400 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3962
[LightGBM] [Info] Number of data points in the train set: 5747365, number of used features: 34
[LightGBM] [Info] Start training from score 0.957398
Training until validation scores don't improve for 50 rounds
[100]	training's rmse: 1.95065	valid_1's rmse: 1.78078
[200]	training's rmse: 1.89511	valid_1's rmse: 1.76738
Early stopping, best iteration is:
[162]	training's rmse: 1.91383	valid_1's rmse: 1.76393


In [67]:
val_pred = model.predict(x_val[features])
val_score = np.sqrt(metrics.mean_squared_error(val_pred, y_val))
print(f'Our val rmse score is {val_score}')

Our val rmse score is 1.7639337947073535


In [68]:
y_pred = model.predict(test[features])
test['demand'] = y_pred

In [69]:
predictions = test[['id', 'date', 'demand']]
predictions = pd.pivot(predictions, index = 'id', columns = 'date', values = 'demand').reset_index()
predictions.columns = ['id'] + ['F' + str(i + 1) for i in range(28)]

In [70]:
evaluation_rows = [row for row in submission['id'] if 'evaluation' in row] 
evaluation = submission[submission['id'].isin(evaluation_rows)]

validation = submission[['id']].merge(predictions, on = 'id')

In [71]:
final = pd.concat([validation, evaluation])
final.to_csv('tx1.csv', index = False)